# Exploratory Data Analysis


_Jerel Novick_

---



In [42]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pandas_profiling as pp
import scipy.stats as stats
import numpy as np
from datetime import datetime, timedelta
#import tabulatehelper as th
%qtconsole
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 9999)
pd.set_option('display.max_rows', 200)

In [2]:
#Read in the master dataset
nba_data = pd.read_csv('./data/2012-18_officialBoxScore.csv')

In [3]:
nba_new = nba_data[['gmDate','teamAbbr','teamRslt','opptAbbr','opptRslt']]

In [4]:
nba_new.insert(3, "teamPtsFor", 0)
nba_new.insert(4, "teamPtsAgnst", 0)
nba_new.insert(7, "opptPtsFor", 0)
nba_new.insert(8, "opptPtsAgnst", 0)

In [5]:
nba_new['gmDate'] = pd.to_datetime(nba_new['gmDate'])

In [6]:
#converted the teamRsl and opptRslt columns from Win and Loss to O for Loss and 1 for Win.
#converted the teamLoc and opptLoc columns from Home and Awaay to O for Away and 1 for Home.
nba_new['teamRslt'] = nba_new['teamRslt'].apply(lambda x: 0 if x=='Loss' else 1)
nba_new['opptRslt'] = nba_new['opptRslt'].apply(lambda x: 0 if x=='Loss' else 1)

In [7]:
nba_new.head()

,gmDate,teamAbbr,teamRslt,teamPtsFor,teamPtsAgnst,opptAbbr,opptRslt,opptPtsFor,opptPtsAgnst
0,2012-10-30,WAS,0,0,0,CLE,1,0,0
1,2012-10-30,WAS,0,0,0,CLE,1,0,0
2,2012-10-30,WAS,0,0,0,CLE,1,0,0
3,2012-10-30,CLE,1,0,0,WAS,0,0,0
4,2012-10-30,CLE,1,0,0,WAS,0,0,0


In [8]:
nba_new = nba_new.drop_duplicates()

In [9]:
nba_new.head()

,gmDate,teamAbbr,teamRslt,teamPtsFor,teamPtsAgnst,opptAbbr,opptRslt,opptPtsFor,opptPtsAgnst
0,2012-10-30,WAS,0,0,0,CLE,1,0,0
3,2012-10-30,CLE,1,0,0,WAS,0,0,0
6,2012-10-30,BOS,0,0,0,MIA,1,0,0
9,2012-10-30,MIA,1,0,0,BOS,0,0,0
12,2012-10-30,DAL,1,0,0,LAL,0,0,0


In [10]:
nba_new = nba_new.reset_index(drop=True)

In [11]:
nba_new.head()

,gmDate,teamAbbr,teamRslt,teamPtsFor,teamPtsAgnst,opptAbbr,opptRslt,opptPtsFor,opptPtsAgnst
0,2012-10-30,WAS,0,0,0,CLE,1,0,0
1,2012-10-30,CLE,1,0,0,WAS,0,0,0
2,2012-10-30,BOS,0,0,0,MIA,1,0,0
3,2012-10-30,MIA,1,0,0,BOS,0,0,0
4,2012-10-30,DAL,1,0,0,LAL,0,0,0


In [12]:
nba_new = nba_new.iloc[::2,:]
nba_new = nba_new.reset_index(drop=True)

In [13]:
nba_new.head()

,gmDate,teamAbbr,teamRslt,teamPtsFor,teamPtsAgnst,opptAbbr,opptRslt,opptPtsFor,opptPtsAgnst
0,2012-10-30,WAS,0,0,0,CLE,1,0,0
1,2012-10-30,BOS,0,0,0,MIA,1,0,0
2,2012-10-30,DAL,1,0,0,LAL,0,0,0
3,2012-10-31,DEN,0,0,0,PHI,1,0,0
4,2012-10-31,IND,1,0,0,TOR,0,0,0


In [15]:
nba_stand = pd.read_csv('./data/2012-18_standings.csv')
nba_stand['stDate'] = pd.to_datetime(nba_stand['stDate'])

In [33]:
nba_stand.head()

,stDate,teamAbbr,rank,rankOrd,gameWon,gameLost,stk,stkType,stkTot,gameBack,ptsFor,ptsAgnst,homeWin,homeLoss,awayWin,awayLoss,confWin,confLoss,lastFive,lastTen,gamePlay,ptsScore,ptsAllow,ptsDiff,opptGmPlay,opptGmWon,opptOpptGmPlay,opptOpptGmWon,sos,rel%Indx,mov,srs,pw%,pyth%13.91,wpyth13.91,lpyth13.91,pyth%16.5,wpyth16.5,lpyth16.5
0,2012-10-30,ATL,3,3rd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
1,2012-10-30,BKN,3,3rd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
2,2012-10-30,BOS,14,14th,0,1,L1,loss,1,1.0,107,120,0,0,0,1,0,1,0,0,1,107.0,120.0,-13.0,0,0,0,0,0.0,0.0,-13.0,-13.0,0.072,0.1687,13.8334,68.1666,0.131,10.742,71.258
3,2012-10-30,CHA,3,3rd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
4,2012-10-30,CHI,3,3rd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000


In [16]:
nba_stand.insert(1, "gmDate",0)

In [17]:
nba_stand

,stDate,gmDate,teamAbbr,rank,rankOrd,gameWon,gameLost,stk,stkType,stkTot,gameBack,ptsFor,ptsAgnst,homeWin,homeLoss,awayWin,awayLoss,confWin,confLoss,lastFive,lastTen,gamePlay,ptsScore,ptsAllow,ptsDiff,opptGmPlay,opptGmWon,opptOpptGmPlay,opptOpptGmWon,sos,rel%Indx,mov,srs,pw%,pyth%13.91,wpyth13.91,lpyth13.91,pyth%16.5,wpyth16.5,lpyth16.5
0,2012-10-30,0,ATL,3,3rd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0000,0.000000,0.0000,0.0000,0.5000,0.0000,0.0000,82.0000,0.0000,0.0000,82.0000
1,2012-10-30,0,BKN,3,3rd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0000,0.000000,0.0000,0.0000,0.5000,0.0000,0.0000,82.0000,0.0000,0.0000,82.0000
2,2012-10-30,0,BOS,14,14th,0,1,L1,loss,1,1.0,107,120,0,0,0,1,0,1,0,0,1,107.0,120.0,-13.0,0,0,0,0,0.0000,0.000000,-13.0000,-13.0000,0.0720,0.1687,13.8334,68.1666,0.1310,10.7420,71.2580
3,2012-10-30,0,CHA,3,3rd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0000,0.000000,0.0000,0.0000,0.5000,0.0000,0.0000,82.0000,0.0000,0.0000,82.0000
4,2012-10-30,0,CHI,3,3rd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0000,0.000000,0.0000,0.0000,0.5000,0.0000,0.0000,82.0000,0.0000,0.0000,82.0000
5,2012-10-30,0,CLE,1,1st,1,0,W1,win,1,0.0,94,84,1,0,0,0,1,0,1,1,1,94.0,84.0,10.0,0,0,0,0,0.0000,0.000000,10.0000,10.0000,0.8293,0.8270,67.8140,14.1860,0.8648,70.9136,11.0864
6,2012-10-30,0,DAL,1,1st,1,0,W1,win,1,0.0,99,91,0,0,1,0,1,0,1,1,1,99.0,91.0,8.0,0,0,0,0,0.0000,0.000000,8.0000,8.0000,0.7634,0.7635,62.6070,19.3930,0.8006,65.6492,16.3508
7,2012-10-30,0,DEN,2,2nd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0000,0.000000,0.0000,0.0000,0.5000,0.0000,0.0000,82.0000,0.0000,0.0000,82.0000
8,2012-10-30,0,DET,3,3rd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0000,0.000000,0.0000,0.0000,0.5000,0.0000,0.0000,82.0000,0.0000,0.0000,82.0000
9,2012-10-30,0,GS,2,2nd,0,0,-,-,0,0.5,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0000,0.000000,0.0000,0.0000,0.5000,0.0000,0.0000,82.0000,0.0000,0.0000,82.0000


In [74]:
len(nba_new)

7379

In [36]:
nba_stand['gmDate'] = pd.to_datetime(nba_stand['gmDate'])

In [ ]:
nba_stand.insert(1, "month", 0)

In [ ]:
for g in range(0, len(nba_stand)):
    nba_stand['gmDate'] = (nba_stand['stDate'] + timedelta(days=1))

In [18]:
nba_stand['gmDate'] = (nba_stand['stDate'] + timedelta(days=1))

In [24]:
nba_new.head(15)

,gmDate,teamAbbr,teamRslt,teamPtsFor,teamPtsAgnst,opptAbbr,opptRslt,opptPtsFor,opptPtsAgnst
0,2012-10-30,WAS,0,0,0,CLE,1,0,0
1,2012-10-30,BOS,0,0,0,MIA,1,0,0
2,2012-10-30,DAL,1,0,0,LAL,0,0,0
3,2012-10-31,DEN,0,0,0,PHI,1,0,0
4,2012-10-31,IND,1,0,0,TOR,0,0,0
5,2012-10-31,HOU,1,0,0,DET,0,0,0
6,2012-10-31,SAC,0,0,0,CHI,1,0,0
7,2012-10-31,SA,1,0,0,NO,0,0,0
8,2012-10-31,DAL,0,0,0,UTA,1,0,0
9,2012-10-31,GS,1,0,0,PHO,0,0,0


In [21]:
nba_stand = nba_stand[['gmDate','teamAbbr', 'ptsFor','ptsAgnst']]

In [23]:
nba_stand.head(15)

,gmDate,teamAbbr,ptsFor,ptsAgnst
0,2012-10-31,ATL,0,0
1,2012-10-31,BKN,0,0
2,2012-10-31,BOS,107,120
3,2012-10-31,CHA,0,0
4,2012-10-31,CHI,0,0
5,2012-10-31,CLE,94,84
6,2012-10-31,DAL,99,91
7,2012-10-31,DEN,0,0
8,2012-10-31,DET,0,0
9,2012-10-31,GS,0,0


In [32]:
nba_stand.to_csv('./data/nba_stand.csv', index=False)

In [33]:
nba_new.to_csv('./data/nba_new.csv', index=False)

In [39]:
nba_stand.head(10).to_clipboard(sep=',', index=True)

In [18]:
for s in range(0, len(nba_stand)):
    for d in range(0, len(nba_new)):
        if (nba_stand.iloc[s,0] + timedelta(days=1)) == (nba_new.iloc[d,0]) and nba_stand.iloc[s,1] == nba_new.iloc[d,1]:
            nba_new.iloc[d,3] = nba_stand.iloc[s,10]
            nba_new.iloc[d,4] = nba_stand.iloc[s,11]

KeyboardInterrupt: 

In [ ]:
for s in range(0, len(nba_stand)):
    for d in range(0, len(nba_new)):
        if (nba_stand.iloc[s,0] + timedelta(days=1)) == (nba_new.iloc[d,0]) and nba_stand.iloc[s,1] == nba_new.iloc[d,5]:
            nba_new.iloc[d,7] = nba_stand.iloc[s,10]
            nba_new.iloc[d,8] = nba_stand.iloc[s,11]